In [1]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=4, n_workers=2)
client

Client Scheduler: tcp://127.0.0.1:45993 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 8 Memory: 503.05 GiB


In [57]:
client.cluster.scale(50)

In [58]:
client

Client Scheduler: tcp://127.0.0.1:38603 Dashboard: http://127.0.0.1:38417/status,Cluster Workers: 50 Cores: 200 Memory: 540.15 GB


___

In [ ]:
conda install -c conda-forge dask=2021.4.1 dask-core=2021.4.1 distributed=2021.4.1 dask-yarn

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::dask-mpi==2.21.0=py38h414b51d_1
failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
Examining conflict for jupyter_console scipy pyopenssl webencodings greenlet so| 

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from scipy import interpolate
import pickle
import dask

In [3]:
def inc(x):
    return x+1

In [4]:
L = client.map(inc, range(10))

ImportError: cannot import name 'dumps_msgpack' from 'distributed.protocol.core' (/usr/common/software/python/3.8-anaconda-2021.05/lib/python3.8/site-packages/distributed/protocol/core.py)

In [16]:
import time
import random

def costly_simulation(list_param):
    time.sleep(random.random())
    return sum(list_param)

In [17]:
import dask
lazy_results = []

In [19]:
import pandas as pd
import numpy as np

input_params = pd.DataFrame(np.random.random(size=(500, 4)),
                            columns=['param_a', 'param_b', 'param_c', 'param_d'])
input_params.head()

,param_a,param_b,param_c,param_d
0,0.727540,0.533437,0.489232,0.948024
1,0.958734,0.585661,0.085179,0.621185
2,0.615474,0.637609,0.425324,0.208987
3,0.057558,0.692922,0.048415,0.966129
4,0.070456,0.440332,0.946259,0.823255


In [20]:
%%time

for parameters in input_params.values[:10]:
    lazy_result = dask.delayed(costly_simulation)(parameters)
    lazy_results.append(lazy_result)

CPU times: user 3.16 ms, sys: 29 µs, total: 3.18 ms
Wall time: 2.16 ms


In [22]:
lazy_results[0]

Delayed('costly_simulation-72b8aba5-f078-4a24-bee9-9c3b3efc45ae')

In [21]:
%time dask.compute(*lazy_results)

ImportError: cannot import name 'dumps_msgpack' from 'distributed.protocol.core' (/usr/common/software/python/3.8-anaconda-2021.05/lib/python3.8/site-packages/distributed/protocol/core.py)

In [3]:
#Read in raw data Interp_load and TimeRange_load.
#Interp_load is a dictionary with 6 keys (bands in our case), 
#the item of each of them is a list that contains 40000 functions interpolated with scipy interp1d.
#TimeRange_load tells their domains of definition.

FilePath = '/global/homes/l/lianming/Presto-Color-2/data/2Day_Interp/SNIa-SALT2_Interp.pkl'

with open(FilePath, 'rb') as f:
    Interp_load = pickle.load(f)
    TimeRange_load = pickle.load(f)  

In [4]:
Interp_load['u'][0]

In [28]:
Interp_load = {key: Interp_load[key][0:10] for key in Interp_load.keys()}
TimeRange_load = {key: TimeRange_load[key][0:10] for key in TimeRange_load.keys()}

In [29]:
def CalculateMap(dT1, dT2=300, 
                 Interp_load=Interp_load, TimeRange_load=TimeRange_load, 
                 Band1='g', Band2='i',  
                 PointsPDay = 1, Thr=27.5, ObjNo=10):
        
    dMag = []
    Color = []
    
    if ObjNo == None:
        ObjNo = len(Interp_load[Band1])

    for II in range(ObjNo):
        
        if Interp_load[Band1][II]==[] or Interp_load[Band2][II]==[]:
            continue
            
        TimeRangeStart = max( TimeRange_load[Band1][II][0], TimeRange_load[Band2][II][0] - dT1/1440 )
        TimeRangeEnd = min( TimeRange_load[Band1][II][1] - dT2/1440, TimeRange_load[Band2][II][1] - dT1/1440 )

        TimeRange = TimeRangeEnd - TimeRangeStart
        SampleNo = np.int(PointsPDay*TimeRange)
        
        XX = np.random.rand(SampleNo)*TimeRange + TimeRangeStart

        Mag1 = Interp_load[Band1][II](XX)
        Mag2 = Interp_load[Band2][II](XX+dT1/1440)
        Mag12 = Interp_load[Band1][II](XX+dT2/1440)

        Mask = (Mag1<Thr) * (Mag2<Thr) *(Mag12<Thr)

        dMag.extend(Mag1[Mask] - Mag12[Mask])
        Color.extend(Mag1[Mask] - Mag2[Mask])
        
    data = np.array([dMag, Color])
    
    return data

In [21]:
def reduce(results):
    
    dMags = []
    Colors = []
    
    for dMag, Color in results:
        
#         print(len(dMag), len(Color))
        dMags.extend(dMag)
        Colors.extend(Color)
    print(len(dMags), len(Colors))
    return np.array([dMags, Colors])

In [16]:
dT1s = np.arange(0,60, 15)

In [17]:
dT1s

array([ 0, 15, 30, 45])

---
When run directly.

In [22]:
%%time
map = CalculateMap(dT1=30, dT2=300, Interp_load=Interp_load, TimeRange_load=TimeRange_load, Band1='g', Band2='i', PointsPDay=0.1)

CPU times: user 7.24 ms, sys: 0 ns, total: 7.24 ms
Wall time: 5.63 ms


---
Try with delayed and compute